In [1]:
import tensorflow as tf

name = 'alice'
in_length = 8  # mesage of length 4, so expect output of length 4
conv_params = [
    [[4, 1, 2], 1], 
    [[2, 2, 4], 2], 
    [[1, 4, 4], 1], 
    [[1, 4, 1], 1]
]

In [2]:
class Net:
    def __init__(self, name, in_length, conv_params):
        self.name = name  # a string
        self.conv_params = conv_params  # format: [filter_shape, stride]
        self.in_length = in_length
        self.fc_weights = tf.get_variable(
            name=self.name + '_fc_weights', shape=[in_length, in_length],
            initializer=tf.contrib.layers.xavier_initializer()
        )  # do we want to include biases? others don't
        self.conv_weights = [tf.get_variable(
            self.name + 'conv_weights'+str(conv_params.index(param)),
            shape=param[0], initializer=tf.contrib.layers.xavier_initializer()
        )
                             for param in self.conv_params]

    def fc_layer(self, in_tensor):
        # input: a tensor.
        # if we generate inputs as lists of binary
        # digits like [1,0,1,1,...], then process
        # as tf.Variable(binary_message, dtype=tf.float32)

        # in_tensor is one of
        #     plaintext + key for alice
        #     ciphertext + key for bob
        #     ciphertext for eve

        in_tensor = tf.expand_dims(in_tensor, 1)
        return tf.nn.sigmoid(tf.matmul(self.fc_weights, in_tensor))

    def conv_layer(self, in_tensor):
        # input: a tensor of shape [in_length, 1]
        # conv1d needs two 3-dimensional tensors as as input
        out_tensor = tf.expand_dims(in_tensor, 0)

        # for all but the last layers we use relu
        for weights in self.conv_weights[:-1]:
            stride = self.conv_params[self.conv_weights.index(weights)][1]
            # dictionary would be nicer
            out_tensor = tf.nn.relu(
                tf.nn.conv1d(out_tensor, weights, stride, padding='SAME')
            )  # if problems later, maybe I should have used placeholders?

        # for the last layer use a tanh
        weights = self.conv_weights[-1]
        stride = self.conv_params[self.conv_weights.index(weights)][1]
        out_tensor = tf.nn.tanh(
           tf.nn.conv1d(out_tensor, self.conv_weights[-1], stride, padding='SAME')
        )  # if problems later, maybe I should have used placeholders?

        out_tensor = tf.squeeze(out_tensor)

        # now round to 0's and 1's
        # warning: was getting -0.'s instead of 0's. prob ok but...
        out_tensor = tf.ceil(out_tensor)

        return out_tensor
    
    def loss_func(self, name, plaintext, eve_output, bob_output):
        # All Nets need to use Eve's loss function
        # Eve uses it as a loss function and Alice/Bob use it to build their loss func
        
        # This should be the L1-Loss of the original plaintext and what Eve decrypted
        eveLoss_L1 = tf.reduce_sum(tf.abs(plaintext - eve_output))
        
        # Then, perform the modification Abadi & Andersen describe in 2.5 of (N/2 - Eve_L1)^2 / (N/2)^2
        # This should cause the network to drive Eve towards a 50% bit error rate
        eveLoss = tf.pow((plaintext.get_shape().as_list()[0]/2 - eveLoss_L1), 2) / tf.pow((plaintext.get_shape().as_list()[0])/2, 2)
        
        # Alice & Bob use the same loss function
        if self.name == 'alice' or self.name == 'bob':
            # Alice and Bob's loss is the L1-loss of [originalPlaintext, key] and what Bob recovered minus eveLoss
            aliceBobLoss = tf.reduce_sum(tf.abs(bob_output - plaintext)) - eveLoss
            return aliceBobLoss
        else:
            return eveLoss

In [3]:
A = Net(name, in_length, conv_params)

B = Net('bob', in_length, conv_params)

sess = tf.Session()

In [4]:
in_tensor = [1,0,1,0,1,0,1,0]
in_tensor = tf.Variable(in_tensor, dtype=tf.float32)
sess.run(tf.global_variables_initializer())

In [5]:
fc_output = A.fc_layer(in_tensor)
sess.run(fc_output)

array([[ 0.43095133],
       [ 0.44031957],
       [ 0.65459198],
       [ 0.35066968],
       [ 0.35429078],
       [ 0.470884  ],
       [ 0.25281075],
       [ 0.54090756]], dtype=float32)

In [6]:
conv_output = A.conv_layer(fc_output)
sess.run(conv_output)

array([ 1.,  0.,  0.,  0.], dtype=float32)

In [7]:
bob_input = tf.concat(0, [conv_output, in_tensor[-4:]])
sess.run(bob_input)

array([ 1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.], dtype=float32)

In [8]:
bob_output = B.conv_layer(B.fc_layer(bob_input))
sess.run(bob_output)

array([ 0., -0.,  0.,  1.], dtype=float32)

In [9]:
# In this setup, eve_output has an error of (N/2 - Eve_L1)^2 / (N/2)^2 = (2 - 4)^2 / (2)^2 = 1

# Thus, the total bob_loss should be whatever the L1 difference is of bob_output and plaintext (in this case, 2) and subtract 1
bob_loss = B.loss_func(name='bob', plaintext=in_tensor[0:4],
                       eve_output=[0, 1, 0, 1], bob_output=bob_output)
sess.run(bob_loss)

2.0